In [6]:
# Import packages

import pandas as pd 
import numpy as np 
import os

pd.set_option('display.max_columns', None)

In [2]:
five_thirty_eight = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')

In [16]:
# Only need recent data, and only need certain columns

fte = five_thirty_eight.copy()
fte = fte[fte.season >= 2014]
fte = fte[['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'qb1', 'qb2', 'score1', 'score2',
          'qb1_value_pre', 'qb2_value_pre', 'qbelo_prob1', 'qbelo_prob2']]

fte = fte.rename(columns = {
    'team1': 'home',
    'team2': 'away',
    'score1': 'home_score',
    'score2': 'away_score',
    'qb1': 'home_qb',
    'qb2': 'away_qb'
})

In [27]:
# fte[(fte.home_score.notnull()) & ((fte.home == 'LAR') | (fte.away == ''))].tail(20)

In [30]:
# Fix team names

def fix_team_names(game, home=True):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team'
    }
    
    if home:
        return team_mapping[game['home']]
    
    else:
        return team_mapping[game['away']]
    
fte['home'] = fte.apply(lambda x: fix_team_names(x, home=True), axis=1)
fte['away'] = fte.apply(lambda x: fix_team_names(x, home=False), axis=1)

In [32]:
# Remove games that have not occured yet

fte = fte[fte.home_score.notnull()]

In [51]:
# Save data frame

fte.to_csv('../data/five_thirty_eight_cleaned.csv')

In [52]:
# Get data frame with starting qb info

starting_qbs = fte[['date', 'season', 'home', 'away', 'home_score', 'away_score', 'home_qb', 'away_qb']]

def get_abbreviated_qb(qb):
    qb_split = qb.split()
    first = qb_split[0][0] + '.'
    last = qb_split[-1]
    qb_full = first + ' ' + last
    return qb_full

starting_qbs['home_qb_abv'] = starting_qbs.apply(lambda x: get_abbreviated_qb(x.home_qb), axis=1)
starting_qbs['away_qb_abv'] = starting_qbs.apply(lambda x: get_abbreviated_qb(x.away_qb), axis=1)

starting_qbs.head()

<ipython-input-52-d229ca1aa500>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_qbs['home_qb_abv'] = starting_qbs.apply(lambda x: get_abbreviated_qb(x.home_qb), axis=1)
<ipython-input-52-d229ca1aa500>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_qbs['away_qb_abv'] = starting_qbs.apply(lambda x: get_abbreviated_qb(x.away_qb), axis=1)


,date,season,home,away,home_score,away_score,home_qb,away_qb,home_qb_abv,away_qb_abv
14939,2014-09-04,2014,Seattle Seahawks,Green Bay Packers,36.0,16.0,Russell Wilson,Aaron Rodgers,R. Wilson,A. Rodgers
14940,2014-09-07,2014,Denver Broncos,Indianapolis Colts,31.0,24.0,Peyton Manning,Andrew Luck,P. Manning,A. Luck
14941,2014-09-07,2014,Philadelphia Eagles,Jacksonville Jaguars,34.0,17.0,Nick Foles,Chad Henne,N. Foles,C. Henne
14942,2014-09-07,2014,Kansas City Chiefs,Tennessee Titans,10.0,26.0,Alex Smith,Jake Locker,A. Smith,J. Locker
14943,2014-09-07,2014,Los Angeles Rams,Minnesota Vikings,6.0,34.0,Shaun Hill,Matt Cassel,S. Hill,M. Cassel


In [50]:
starting_qbs.to_csv('../data/starting_qbs.csv')